In [1]:
import sqlite3
from datetime import datetime


class DatabaseManager:
    def __init__(self, db_name="library.db"):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS books (
            book_id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            author TEXT NOT NULL,
            status TEXT DEFAULT 'Available',
            issued_date TEXT
        )
        """)
        self.conn.commit()


class LibraryService:
    def __init__(self, db_manager):
        self.db = db_manager

    def add_book(self, title, author):
        self.db.cursor.execute("""
        INSERT INTO books (title, author)
        VALUES (?, ?)
        """, (title, author))
        self.db.conn.commit()
        return "Book added successfully."

    def get_all_books(self):
        self.db.cursor.execute("SELECT * FROM books")
        return self.db.cursor.fetchall()

    def issue_book(self, book_id):
        self.db.cursor.execute(
            "SELECT status FROM books WHERE book_id = ?", (book_id,)
        )
        book = self.db.cursor.fetchone()

        if not book:
            return "Invalid Book ID."

        if book[0] == "Issued":
            return "Book already issued."

        issue_date = datetime.now().strftime("%Y-%m-%d")
        self.db.cursor.execute("""
        UPDATE books
        SET status = 'Issued', issued_date = ?
        WHERE book_id = ?
        """, (issue_date, book_id))
        self.db.conn.commit()
        return "Book issued successfully."

    def return_book(self, book_id):
        self.db.cursor.execute("""
        UPDATE books
        SET status = 'Available', issued_date = NULL
        WHERE book_id = ?
        """, (book_id,))
        self.db.conn.commit()
        return "Book returned successfully."


class LibraryUI:
    def __init__(self):
        self.db_manager = DatabaseManager()
        self.library_service = LibraryService(self.db_manager)

    def display_books(self):
        books = self.library_service.get_all_books()
        print("\nID | Title | Author | Status | Issued Date")
        print("-" * 50)
        for book in books:
            print(book)

    def start(self):
        while True:
            print("\n--- Library Management System ---")
            print("1. Add Book")
            print("2. View Books")
            print("3. Issue Book")
            print("4. Return Book")
            print("5. Exit")

            choice = input("Enter choice: ")

            if choice == "1":
                title = input("Enter title: ")
                author = input("Enter author: ")
                print(self.library_service.add_book(title, author))

            elif choice == "2":
                self.display_books()

            elif choice == "3":
                book_id = int(input("Enter Book ID: "))
                print(self.library_service.issue_book(book_id))

            elif choice == "4":
                book_id = int(input("Enter Book ID: "))
                print(self.library_service.return_book(book_id))

            elif choice == "5":
                print("Exiting application.")
                break

            else:
                print("Invalid option.")


if __name__ == "__main__":
    app = LibraryUI()
    app.start()



--- Library Management System ---
1. Add Book
2. View Books
3. Issue Book
4. Return Book
5. Exit
Enter choice: 6
Invalid option.

--- Library Management System ---
1. Add Book
2. View Books
3. Issue Book
4. Return Book
5. Exit
Enter choice: 2

ID | Title | Author | Status | Issued Date
--------------------------------------------------

--- Library Management System ---
1. Add Book
2. View Books
3. Issue Book
4. Return Book
5. Exit
Enter choice: 4
Enter Book ID: 1
Book returned successfully.

--- Library Management System ---
1. Add Book
2. View Books
3. Issue Book
4. Return Book
5. Exit
Enter choice: 5
Exiting application.
